<a href="https://colab.research.google.com/github/ameasure/colab_tutorials/blob/master/Finetune2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transfer Learning with Finetune 

https://https://github.com/IndicoDataSolutions/finetune

In [1]:

!pip install -U finetune
#!python -m spacy download en_core_web_sm
#!python -m spacy link en_core_web_sm en
#!pip install tables
#!pip install xlrd
!wget 'https://github.com/ameasure/autocoding-class/raw/master/msha.xlsx'

Requirement already up-to-date: finetune in /usr/local/lib/python3.6/dist-packages (0.6.7)
--2019-06-16 15:59:33--  https://github.com/ameasure/autocoding-class/raw/master/msha.xlsx
Resolving github.com (github.com)... 13.250.177.223
Connecting to github.com (github.com)|13.250.177.223|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ameasure/autocoding-class/master/msha.xlsx [following]
--2019-06-16 15:59:33--  https://raw.githubusercontent.com/ameasure/autocoding-class/master/msha.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4183086 (4.0M) [application/octet-stream]
Saving to: ‘msha.xlsx.2’

msha.xlsx.2         100%[===================>]   3.99M  --.-KB/s    in 0.03s   

2019-06-16 15:59:34

In [2]:
import pandas as pd

df = pd.read_excel('msha.xlsx')
df['ACCIDENT_YEAR'] = df['ACCIDENT_DT'].apply(lambda x: x.year)
df['ACCIDENT_YEAR'].value_counts()
df_train = df[df['ACCIDENT_YEAR'].isin([2010, 2011])][:3200].copy()
df_valid = df[df['ACCIDENT_YEAR'] == 2012][:1000].copy()
print('training rows:', len(df_train))
print('validation rows:', len(df_valid))

training rows: 3200
validation rows: 1000


In [3]:
import math

batch_size = 32
val_size = 320
train_size = len(df_train) - val_size
# validate once every epoch
val_interval = math.floor(train_size / batch_size)
print(f'val_interval={val_interval}')

val_interval=90


In [4]:
from finetune import Classifier

# max_length is the maximum number of words we will use from each narrative
# behind the scenes finetune is setting aside 5% of the training data for validation
model = Classifier(batch_size=batch_size, 
                   max_length=90, 
                   n_epochs=3, 
                   val_size=val_size,
                   val_interval=val_interval,
                   eval_acc=True)
model.fit(df_train['NARRATIVE'], df_train['INJ_BODY_PART'])

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
W0616 15:59:40.769783 140626639210368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/finetune/saver.py:66: The name tf.train.SessionRunHook is deprecated. Please use tf.estimator.SessionRunHook instead.

W0616 15:59:40.780952 140626639210368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/finetune/__init__.py:23: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0616 15:59:40.782217 140626639210368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/finetune/__init__.py:23: The name tf.logging.ERROR is deprecated. Please use t

In [0]:
from finetune import Classifier
from sklearn.metrics import accuracy_score, f1_score

# max_length is the maximum number of words we will use from each narrative
model = Classifier(batch_size=batch_size, 
                   max_length=90, 
                   val_size=0,
                   n_epochs=1,
                   eval_acc=True)

N_EPOCHS = 5
for epoch in range(N_EPOCHS):
  model.fit(df_train['NARRATIVE'], df_train['INJ_BODY_PART'])
  with model.cached_predict():
    # generate some predictions on the validation data and see where we are
    preds = model.predict(df_valid['NARRATIVE'].values)
    acc = accuracy_score(y_true=df_valid['INJ_BODY_PART'], y_pred=preds)
    print(f'Epoch {epoch}: val_accuracy={acc}')


I0616 16:16:30.611967 140626639210368 base.py:104] Saving tensorboard output to /tmp/Finetuneug2dwhvd
Epoch 3/3:  88%|████████▊ | 2816/3200 [00:56<00:17, 21.50it/s]

In [5]:
# generate preditions
df_valid['PREDICTED_PART'] = model.predict(df_valid['NARRATIVE'].values)
# look at a sample
df_valid[['NARRATIVE', 'INJ_BODY_PART', 'PREDICTED_PART']].sample(5)

Inference:   0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/finetune/encoding/input_encoder.py:182: UserWarning: Some examples are longer than the max_length. Please trim documents or increase `max_length`. Fallback behaviour is to use the first 88 byte-pair encoded tokens
  "Fallback behaviour is to use the first {} byte-pair encoded tokens".format(max_length - 2)
/usr/local/lib/python3.6/dist-packages/finetune/encoding/input_encoder.py:135: UserWarning: Document is longer than max length allowed, trimming document to 90 tokens.
  max_length
Inference: 100%|██████████| 1000/1000 [00:17<00:00, 58.60it/s]


,NARRATIVE,INJ_BODY_PART,PREDICTED_PART
2486,Coal rolled out from rib striking employee on ...,LOWER LEG/TIBIA/FIBULA,LOWER LEG/TIBIA/FIBULA
3452,Employee was using a knife to cut a rope and s...,FINGER(S)/THUMB,FINGER(S)/THUMB
3457,Walking on belt line said too much weight on r...,BACK (MUSCLES/SPINE/S-CORD/TAILBONE),MULTIPLE PARTS (MORE THAN ONE MAJOR)
3106,Employee was welding and a foreign body enter ...,EYE(S) OPTIC NERVE/VISON,EYE(S) OPTIC NERVE/VISON
1984,This accident is still under investigation and...,BACK (MUSCLES/SPINE/S-CORD/TAILBONE),BACK (MUSCLES/SPINE/S-CORD/TAILBONE)


In [6]:
pd.options.display.max_colwidth=500

df_valid['PROB_DICT'] = model.predict_proba(df_valid['NARRATIVE'].values)
df_valid[['NARRATIVE', 'PREDICTED_PART', 'PROB_DICT']].head(2)

Inference:   0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/finetune/encoding/input_encoder.py:182: UserWarning: Some examples are longer than the max_length. Please trim documents or increase `max_length`. Fallback behaviour is to use the first 88 byte-pair encoded tokens
  "Fallback behaviour is to use the first {} byte-pair encoded tokens".format(max_length - 2)
/usr/local/lib/python3.6/dist-packages/finetune/encoding/input_encoder.py:135: UserWarning: Document is longer than max length allowed, trimming document to 90 tokens.
  max_length
Inference: 100%|██████████| 1000/1000 [00:17<00:00, 55.82it/s]


,NARRATIVE,PREDICTED_PART,PROB_DICT
2,"Employee, parked s/c on grade at 16-Block #3 Entry Spad #3868. S/c slid approx. 3' pinning oper. between s/c & rib, employee had set park brake and got off machine to move roof bolter cable.",CHEST (RIBS/BREAST BONE/CHEST ORGNS),"{'ABDOMEN/INTERNAL ORGANS': 0.008977418, 'ANKLE': 0.004162629, 'ARM, MULTIPLE PARTS': 0.0032280472, 'ARM,NEC': 0.017140277, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 0.105413824, 'BODY SYSTEMS': 0.0073694945, 'BRAIN': 0.013623325, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 0.26844206, 'EAR(S) EXTERNAL': 0.002892342, 'EAR(S) INTERNAL & HEARING': 0.0050324653, 'ELBOW': 0.002983887, 'EYE(S) OPTIC NERVE/VISON': 0.050639648, 'FACE, MULTIPLE PARTS': 0.0028115963, 'FACE,NEC': 0.004201914, 'FINGER(S)/THU..."
5,Possible heart attack.,BODY SYSTEMS,"{'ABDOMEN/INTERNAL ORGANS': 0.015624995, 'ANKLE': 0.0008264859, 'ARM, MULTIPLE PARTS': 0.0007294926, 'ARM,NEC': 0.0008198121, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 0.0019328524, 'BODY SYSTEMS': 0.82196, 'BRAIN': 0.0011903689, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 0.09010305, 'EAR(S) EXTERNAL': 0.0007199971, 'EAR(S) INTERNAL & HEARING': 0.0033850586, 'ELBOW': 0.00097084785, 'EYE(S) OPTIC NERVE/VISON': 0.00043858247, 'FACE, MULTIPLE PARTS': 0.0006685693, 'FACE,NEC': 0.0005429107, 'FINGER(S..."
8,Employee was cleaning up plant spillage into a fork lift mounted self-dumping hopper. Latch on hopper was not operating properly and employee repeatedly lifted on hopper to unlatch it. Employee reported that his right shoulder hurt from the lifting at end of shift. Placed on light duty with no heavy lifting.,SHOULDERS (COLLARBONE/CLAVICLE/SCAPULA),"{'ABDOMEN/INTERNAL ORGANS': 7.5786505e-05, 'ANKLE': 7.940103e-06, 'ARM, MULTIPLE PARTS': 7.815971e-05, 'ARM,NEC': 8.496351e-05, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 0.0012391361, 'BODY SYSTEMS': 7.592077e-05, 'BRAIN': 0.00010860118, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 0.00021058852, 'EAR(S) EXTERNAL': 6.309702e-05, 'EAR(S) INTERNAL & HEARING': 8.138579e-05, 'ELBOW': 0.00016119704, 'EYE(S) OPTIC NERVE/VISON': 7.0898466e-05, 'FACE, MULTIPLE PARTS': 1.2719968e-05, 'FACE,NEC': 8.006192e-0..."
12,"Employee was putting a drag on a shuttle car, when he lifted one end and another employee lifted the other end, the drag smashed the end of his middle finger on his left hand. He was taken to the hospital and received 12 stitches in his finger and returned to work.",FINGER(S)/THUMB,"{'ABDOMEN/INTERNAL ORGANS': 2.6681197e-05, 'ANKLE': 3.0205281e-05, 'ARM, MULTIPLE PARTS': 1.5207753e-05, 'ARM,NEC': 2.7002348e-05, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 1.5651445e-05, 'BODY SYSTEMS': 2.9805162e-05, 'BRAIN': 1.5299178e-05, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 2.4254985e-05, 'EAR(S) EXTERNAL': 6.670856e-05, 'EAR(S) INTERNAL & HEARING': 4.7708432e-05, 'ELBOW': 3.380854e-05, 'EYE(S) OPTIC NERVE/VISON': 2.8072725e-05, 'FACE, MULTIPLE PARTS': 5.0589446e-05, 'FACE,NEC': 4.4717..."
27,While using a cutting torch to remove a bearing the employee was burned by the torch tip after another employee tripped over the torch hose causing the torch flame to contact his left hand.,HAND (NOT WRIST OR FINGERS),"{'ABDOMEN/INTERNAL ORGANS': 8.5834836e-05, 'ANKLE': 2.7501443e-05, 'ARM, MULTIPLE PARTS': 0.00014462482, 'ARM,NEC': 0.00035717717, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 4.9885002e-05, 'BODY SYSTEMS': 0.00013822167, 'BRAIN': 5.270675e-05, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 6.319137e-05, 'EAR(S) EXTERNAL': 8.288939e-05, 'EAR(S) INTERNAL & HEARING': 0.00020017583, 'ELBOW': 6.754224e-05, 'EYE(S) OPTIC NERVE/VISON': 0.00028945727, 'FACE, MULTIPLE PARTS': 0.00010168808, 'FACE,NEC': 0.000111..."


In [7]:
# function that takes a row of our dataframe and returns the predicted probability
def get_probability(row):
    predicted_part = row['PREDICTED_PART']
    probability_dict = row['PROB_DICT']
    return probability_dict[predicted_part]

# apply get_probability to each row in our dataframe and store the result
df_valid['PREDICTED_PROB'] = df_valid.apply(func=get_probability, axis=1)
# take a peak at what we get
df_valid[['NARRATIVE', 'INJ_BODY_PART', 'PREDICTED_PART', 'PREDICTED_PROB']].sample(5).head()

,NARRATIVE,INJ_BODY_PART,PREDICTED_PART,PREDICTED_PROB
421,Hearing Loss,EAR(S) INTERNAL & HEARING,EAR(S) INTERNAL & HEARING,0.973974
3249,Employee was performing routine maintenance on the propel motor on shovel 4 when he experienced electric shock.,BODY SYSTEMS,BODY SYSTEMS,0.284030
463,A PIECE OF WATER LINE FELL TWO FEET AND STRUCK EMPLOYEE ON THE SHOULDER CAUSING BRUISING.,SHOULDERS (COLLARBONE/CLAVICLE/SCAPULA),SHOULDERS (COLLARBONE/CLAVICLE/SCAPULA),0.963299
2703,Operator was changing drill steel bit when the chain wrench broke and a piece of the bar hit the back/side of the operators left knee. No initial treatment was sought. Area around the impact began to swell after a couple of days and that's when the operator sought medical attention,KNEE/PATELLA,KNEE/PATELLA,0.878149
3402,Employee slipped and fell into a sump hole injuring shoulder and back. Company was notified this was a medical injury on 12/19/2012.,BACK (MUSCLES/SPINE/S-CORD/TAILBONE),MULTIPLE PARTS (MORE THAN ONE MAJOR),0.803995


In [8]:
from sklearn.metrics import accuracy_score, f1_score

mf1 = f1_score(df_valid['INJ_BODY_PART'], df_valid['PREDICTED_PART'], average='macro')
acc = accuracy_score(df_valid['INJ_BODY_PART'], df_valid['PREDICTED_PART'])
print('macro-f1:', mf1)
print('accuracy:', acc)

macro-f1: 0.5587960487477243
accuracy: 0.808


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
model.generate_text(seed_text='He was cleaning the')

'_start_he was cleaning the mess spills spills spills spills spills spills spill pour pours pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour pour '

In [15]:
model.config()

TypeError: ignored

In [0]:
model.pred